# Notebook para Pré-Processamento de Dados

## Importação das Bibliotecas

In [ ]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import pdfplumber
from unidecode import unidecode

## Load dos dados intermediários

In [ ]:
df = pd.read_csv("suicidios_2010_a_2019.csv")
df

<ipython-input-482-2a9d09f6e074>:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("suicidios_2010_a_2019.csv")


,Unnamed: 0,estado,ano,mes,DTOBITO,DTNASC,SEXO,RACACOR,ASSISTMED,ESCMAE,ESTCIV,ESC,OCUP,CODMUNRES,CAUSABAS,CAUSABAS_O,LOCOCOR,CIRURGIA
0,1,AC,2010,1,2010-01-31,1999-11-09,Masculino,Parda,NaN,NaN,Solteiro/a,1 a 3 anos,ESTUDANTE,Tarauacá,X780,X780,Via pública,Não
1,2,AC,2010,2,2010-02-07,1985-09-23,Masculino,Parda,NaN,NaN,Solteiro/a,1 a 3 anos,ESTUDANTE,Tarauacá,X780,X780,Hospital,Não
2,3,AC,2010,2,2010-02-19,1995-08-06,Masculino,Parda,NaN,NaN,Solteiro/a,1 a 3 anos,ESTUDANTE,Tarauacá,X780,X780,Hospital,Não
3,4,AC,2010,4,2010-04-20,1983-12-25,Feminino,Parda,NaN,NaN,Casado/a,1 a 3 anos,DONA DE CASA,Tarauacá,X780,X780,Domicílio,Não
4,5,AC,2010,7,2010-07-24,1982-05-13,Masculino,Parda,NaN,NaN,Casado/a,NaN,DONA DE CASA,Tarauacá,X720,X720,Domicílio,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112486,112487,TO,2019,9,2019-09-14,1991-08-10,Feminino,Parda,Sim,NaN,Solteiro/a,4 a 7 anos,COZINHEIRO GERAL,Colinas do Tocantins,X700,X700,Outros,NaN
112487,112488,TO,2019,9,2019-09-26,1967-02-17,Feminino,Parda,NaN,NaN,Solteiro/a,4 a 7 anos,DONA DE CASA,Santa fé do Araguaia,X700,X700,Domicílio,NaN
112488,112489,TO,2019,8,2019-08-07,1977-07-20,Masculino,Branca,Não,NaN,Casado/a,8 a 11 anos,COPEIRO,Araguaína,X700,X700,Domicílio,NaN
112489,112490,TO,2019,10,2019-10-04,1978-05-03,Masculino,Parda,Sim,NaN,Casado/a,NaN,NaN,Divinópolis do Tocantins,X729,X729,Hospital,NaN


## Feature Selection

* Durante o pré-processamente, identificou-se que algumas colunas que ainda estavam nos dados intermediários não eram tão pertinentes a solução da problemática. São elas as colunas: DTOBITO, HORAOBITO, LOCOCOR, ASSISTMED.

* As features DTOBITO e HORAOBITO representam informações temporais dos óbitos, enquanto LOCOCOR E ASSISTMED são características de localidade e circustância. Contudo, já que o objetivo do projeto e identificar pessoas vulneráveis ao suícidio para conscientizar e evitar fatalidades, essas informações acabam não contribuindo. Por isso optou-se por removê-las da base.

In [ ]:
df.columns

Index(['Unnamed: 0', 'estado', 'ano', 'mes', 'DTOBITO', 'DTNASC', 'SEXO',
       'RACACOR', 'ASSISTMED', 'ESCMAE', 'ESTCIV', 'ESC', 'OCUP', 'CODMUNRES',
       'CAUSABAS', 'CAUSABAS_O', 'LOCOCOR', 'CIRURGIA'],
      dtype='object')

In [ ]:
df = df.drop(columns=['Unnamed: 0', 'ano', 'mes', 'ASSISTMED', 'ESCMAE', 'CODMUNRES', 'CAUSABAS_O', 'LOCOCOR', 'CIRURGIA'])

## Tratamento de valores nulos e remoção de outliers

* Diversas colunas apresentam valores nulos,simplesmente remover as linhas com essas valores causaria uma grande perda de sinal.
Algumas estratégias devem ser adotadas para preservar a maior qtd. de sinal possível mantendo a maior fidelidade possível com os dados originais.

In [ ]:
df.isnull().sum()

estado          0
DTOBITO         0
DTNASC      10858
SEXO           22
RACACOR      3171
ESTCIV       8700
ESC         28164
OCUP        52217
CAUSABAS        0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112491 entries, 0 to 112490
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   estado    112491 non-null  object
 1   DTOBITO   112491 non-null  object
 2   DTNASC    101633 non-null  object
 3   SEXO      112469 non-null  object
 4   RACACOR   109320 non-null  object
 5   ESTCIV    103791 non-null  object
 6   ESC       84327 non-null   object
 7   OCUP      60274 non-null   object
 8   CAUSABAS  112491 non-null  object
dtypes: object(9)
memory usage: 7.7+ MB


### Escolaridade

* No processo de EDA da base, identificou-se que essa feature é representada pela seguinte legenda:
  * 1: Nenhuma
  * 2: 1 a 3 anos
  * 3: 4 a 7 anos
  * 4: 8 a 11 anos
  * 5: 12 e mais
  * 9: Ignorado

* Para essa feature, pareceu razoável substituir as linhas com valores nulos pela médias dos outros valores da coluna. Contudo, pelo fato dos casos ignorados serem representados por um valor maior que todos os outros (valor igual a 9), o grupo optou por substituí-lo por um valor mais baixo (0) e depois aplicar a média para os nulos.

In [ ]:
# Distribuição dos valores nulos e não-nulos na coluna
# Cabe-se observar a presença de valores iguais a 0, o que não faz parte dos limites encontrados na EDA
# Logo esses valores iguais a 0 são 'outliers' e devem ser descartados
df['ESC'].isnull().sum(), df['ESC'].value_counts()

(28164,
 ESC
 4 a 7 anos     27278
 8 a 11 anos    25947
 1 a 3 anos     15992
 12 e mais       9915
 Nenhuma         5195
 Name: count, dtype: int64)

In [ ]:
 df['ESC'] = df['ESC'].map({'Nenhuma': 1, '1 a 3 anos': 2, '4 a 7 anos': 3, '8 a 11 anos': 4, '12 e mais': 5})

In [ ]:
# Instanciação da classe para substituir os valores nulos
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
# Usa o imputer para substituir os valores nulos pela média
imputer = imputer.fit(df[['ESC']])
df['ESC'] = imputer.transform(df[['ESC']])

In [ ]:
# Distribuição dos valores nulos e não-nulos na coluna
df['ESC'].isnull().sum(), df['ESC'].value_counts()

(0,
 ESC
 3.229998    28164
 3.000000    27278
 4.000000    25947
 2.000000    15992
 5.000000     9915
 1.000000     5195
 Name: count, dtype: int64)

### Estado Civil

* No processo de EDA da base, identificou-se que essa feature é representada pela seguinte legenda:
  * 1: Solteiro
  * 2: Casado
  * 3: Viúvo
  * 4: Separado judicialmente
  * 9: Ignorado

* Para essa feature, adotou-se a estratégia de considerar todos os casos nulos como ignorados (valor igual a 9).

In [ ]:
# Distribuição dos valores nulos e não-nulos na coluna
# Cabe-se observar a presença de valores iguais a 5, o que não faz parte dos limites encontrados na EDA
# Logo esses valores iguais a 5 são 'outliers' e devem ser descartados
df['ESTCIV'].isnull().sum(), df['ESTCIV'].value_counts()

(8700,
 ESTCIV
 Solteiro/a                  56523
 Casado/a                    30001
 Separado/a judicialmente     7771
 União consensual             5280
 Viúvo/a                      4216
 Name: count, dtype: int64)

In [ ]:
 df['ESTCIV'] = df['ESTCIV'].map({'Solteiro/a': 1, 'Casado/a': 2, 'Viúvo/a': 3, 'Separado/a judicialmente': 4, 'União consensual': 9})

In [ ]:
df['ESTCIV'].isnull().sum(), df['ESTCIV'].value_counts()

(8700,
 ESTCIV
 1.0    56523
 2.0    30001
 4.0     7771
 9.0     5280
 3.0     4216
 Name: count, dtype: int64)

In [ ]:
# Considera todos os casos nulos como ignorados
df['ESTCIV'] = df['ESTCIV'].fillna(9)

### Ocupação

* No processo de EDA da base, identificou-se que essa feature representa a ocupação do falecido conforme a Classificação Brasileira de Ocupações (CBO-2002).

* Dito isso, a estratégia adotada foi a de substituir os valores nulos por um valor que já não estivesse presente na coluna e que pudesse caracterizar uma ocupação desconhecida, -1 foi o valor escolhido.

In [ ]:
ocup_map = {
    'desconhecido': -1,
    'estudante': -1,
    'dona de casa': -1,
    'aposentado/pensionista': -1,
    'representante comercial autonomo': 3541,
    'trabalhador agropecuario em geral': 6220,
    'pescador artesanal de agua doce': 5193,
    'professor da educacao do ensino fundamental': 2341,
    'borracheiro': 8485,
    'gari': 5193,
    'produtor agricola polivalente': 6220,
    'caseiro (agricultura)': 6220,
    'garimpeiro': 5193,
    'empregado domestico nos servicos gerais': 5121,
    'carpinteiro': 7711,
    'conferente de carga e descarga': 4141,
    'soldador': 7243,
    'montador de equipamentos eletronicos (maquinas industriais)': 7253,
    'supervisor de almoxarifado': 3541,
    'vendedor ambulante': 5191,
    'eletricista de instalacoes': 9511,
    'desempregado cronico ': -1,
    'operador de colhedor florestal': 6320,
    'cozinheiro geral': 5134,
    'operador de telemarketing ativo': 4223,
    'mecanico de manutencao de automoveis e motocicletas ': 9131,
    'vendedor em domicilio': 5211,
    'motorista de carro de passeio': 7823,
    'garcom': 5134,
    'auxiliar de escritorio, em geral': 4110,
    'biologo': 2121,
    'gesseiro': 7212,
    'gerente comercial': 1421,
    'produtor agropecuario, em geral': 6220,
    'ajudante de motorista': 5191,
    'dirigente do servico publico municipal': 1112,
    'motorista de taxi': 7823,
    'cafeicultor': 6220,
    'comerciante atacadista': 1422,
    'guarda-civil municipal': 5174,
    'professor de nivel medio no ensino fundamental': 2342,
    'ignorada': -1,
    'lustrador de pecas de madeira': 7542,
    'localizador (cobrador)': 3541,
    'marceneiro': 7711,
    'feirante': 5191,
    'carteiro': 5193,
    'trabalhador da cultura de milho e sorgo': 6220,
    'trabalhador na olericultura (frutos e sementes)': 6220,
    'barbeiro': 5161,
    'agente de viagem': 3543,
    'mecanico de manutencao de aeronaves, em geral': 9144,
    'artesao modelador (vidros)': -1,
    'auxiliar de producao farmaceutica': 7842,
    'porteiro (hotel)': 5174,
    'desenhista tecnico': 3185,
    'dirigente do servico publico federal': 1112,
    'encarregado de acabamento de chapas e metais (tempera)': 7212,
    'pescador industrial': 6322,
    'catador de material reciclavel': 5191,
    'passador de roupas em geral': 5163,
    'auxiliar de lavanderia': 5163,
    'ferramenteiro': 7233,
    'cobrador de transportes coletivos (exceto trem)': 7825,
    'alimentador de linha de producao': 7842,
    'operador de monta-cargas (construcao civil)': 7824,
    'atendente de lanchonete': 5134,
    'artista (artes visuais)': 2627,
    'empresario de espetaculo': 1223,
    'medico clinico': 2252,
    'tecnico de manutencao eletronica': 3132,
    'advogado': 2410,
    'professor de disciplinas pedagogicas no ensino medio': 2342,
    'padeiro': 8483,
    'trabalhador da manutencao de edificacoes': 5143,
    'administrador': 2521,
    'arquiteto de edificacoes': 2141,
    'jornalista': 2613,
    'analista de sistemas de automacao': 2122,
    'continuo': 4211,
    'secretaria executiva': 2523,
    'economista': 2515,
    'armador de estrutura de concreto': 7153,
    'auxiliar geral de conservacao de vias permanentes ': 5143,
    'repositor de mercadorias': 5211,
    'trabalhador da pecuaria (bovinos leite)': 6210,
    'auxiliar de servicos de importacao e exportacao': 4110,
    'assistente social': 2516,
    'pedagogo': 2394,
    'professor de ciencias exatas e naturais do ensino fundamental': 2342,
    'tecnico de enfermagem': 3222,
    'assistente de laboratorio industrial': 3116,
    'trabalhador de tratamento do leite ': 8483,
    'frentista': 5191,
    'bobinador eletricista, a mao': 9511,
    'operador de maquinas de construcao civil e mineracao': 7152,
    'ourives': 7243,
    'operador polivalente da industria textil': 7823,
    'abatedor': 7842,
    'operador de caixa': 4211,
    'cabo da policia militar': 5174,
    'mecanico de manutencao de maquinas, em geral': 9144,
    'trabalhador da cultura de arroz': 6210,
    'lavador de veiculos': 5191,
    'tecnico de operacoes e servicos bancarios - renda fixa e variavel': 3135,
    'vaqueador de couros e peles': 5191,
    'fiscal de aviacao civil (fac)': 5112,
    'tecnico agricola': 3112,
    'boiadeiro': 6320,
    'tratorista agricola': 6320,
    'diretor comercial': 1421,
    'seringueiro': 6322,
    'vendedor permissionario': 5211,
    'desossador': 5134,
    'medico cardiologista': 2252,
    'telefonista': 4221,
    'eletrotecnico': 3131,
    'cobrador externo': 4223,
    'agente comunitario de saude': 5151,
    'tecnico em seguranca no trabalho': 3516,
    'operador de escavadeira': 7151,
    'trabalhador de pecuaria polivalente': 6220,
    'funileiro de veiculos (reparacao)': 9144,
    'copiador de chapa': 4122,
    'mestre (construcao civil)': 7151,
    'tecnico em radiologia e imagenologia': 3241,
    'engenheiro civil': 2143,
    'marcador de produtos (siderurgico e metalurgico)': 7243,
    'costureira de reparacao de roupas': 7632,
    'trabalhador de servicos de manutencao de edificios': 5143,
    'cirurgiao dentista - clinico geral': 2234,
    'calceteiro': 5141,
    'professor de lingua inglesa': 2332,
    'atleta profissional (outras modalidades)': 2239,
    'defensor publico': 2410,
    'galvanizador': 7511,
    'montador de moveis e artefatos de madeira': 7711,
    'delegado de policia': 2410,
    'supervisor de carga e descarga': 4102,
    'eletricista de instalacoes (aeronaves)': 3151,
    'empregado domestico faxineiro': 5121,
    'instrutor de auto-escola': 3422,
    'tecelao (tear manual)': 7642,
    'vidraceiro': 7234,
    'agente de seguranca penitenciaria': 5171,
    'refinador de oleo e gordura': 8122,
    'supervisor administrativo': 1210,
    'operador de colheitadeira': 6323,
    'corretor de imoveis': 3541,
    'mecanico de manutencao de aparelhos de climatizacao ': 9532,
    'gerente de logistica (armazenagem e distribuicao)': 1416,
    'almoxarife': 4141,
    'analista financeiro (instituicoes financeiras)': 2514,
    'tecelao (rendas e bordados)': 7633,
    'ajudante de despachante aduaneiro': 5198,
    'tecelao (redes)': 7633,
    'trabalhador da cultura de especiarias': 6110,
    'supervisor da manutencao e reparacao de veiculos leves': 9102,
    'soldador eletrico': 7243,
    'enfermeiro': 2235,
    'tecnico de telecomunicacoes (telefonia)': 3131,
    'moldador, a mao': 7221,
    'vendedor em comercio atacadista': 5211,
    'programador visual grafico': 2614,
    'eletricista de bordo': 3185,
    'costureiro de roupas de couro e pele': 7643,
    'professor de nivel medio na educacao infantil': 2342,
    'pintor de veiculos (reparacao)': 9112,
    'eletricista de manutencao eletroeletronica': 3132,
    'passadeira de pecas confeccionadas': 5162,
    'atendente comercial (agencia postal)': 4110,
    'funileiro industrial': 7233,
    'mecanico de veiculos automotores a diesel (exceto tratores)': 9113,
    'recepcionista, em geral': 4221,
    'pintor de letreiros': 7233,
    'professores de cursos livres': 2394,
    'engraxate': 5199,
    'polidor de metais': 7232,
    'auditor-fiscal da previdencia social': 2411,
    'caminhoneiro autonomo (rotas regionais e internacionais)': 7825,
    'operador de maquina bordatriz': 8112,
    'farmaceutico': 2232,
    'medico veterinario': 2252,
    'embalador, a mao': 7841,
    'baba': 5162,
    'medico anestesiologista': 2251,
    'perito criminal': 2410,
    'operador de maquina de cordoalha': 7242,
    'copeiro': 5134,
    'trabalhador de fabricacao de sorvete': 7841,
    'tecnico de contabilidade': 3541,
    'operador de laminador': 8111,
    'fundidor de metais': 7232,
    'agente funerario': 5199,
    'impressor (serigrafia)': 7731,
    'professor de outras linguas e literaturas': 2394,
    'vendedor pracista': 5211,
    'gerente de producao e operacoes': 1425,
    'auxiliar de corte (preparacao da confeccao de roupas)': 5162,
    'agente publicitario': 3541,
    'atendente de consultorio dentario': 3221,
    'gerente administrativo': 1222,
    'agente de defesa ambiental': 5141,
    'promotor de vendas': 3541,
    'marmorista (construcao)': 7151,
    'fiscal de transportes coletivos (exceto trem)': 4110,
    'coletor de lixo': 5199,
    'musico interprete cantor': 2627,
    'gerente de servicos culturais': 1425,
    'mergulhador profissional (raso e profundo)': 7155,
    'mecanico de manutencao de maquinas cortadoras de grama': 9112,
    'corretor de seguros': 3541,
    'agente de policia federal': 5171,
    'subtenente da policia militar': 5171,
    'engenheiro mecanico': 2142,
    'assentador de canalizacao (edificacoes)': 7151,
    'tecnico em manutencao de equipamentos e instrumentos medico': 3141,
    'trabalhador polivalente da confeccao de calcados': 5162,
    'contramestre de acabamento (industria textil)': 7632,
    'operador de open-end': 7612,
    'professor de ensino superior na area de didatica': 2342,
    'auxiliar de faturamento': 4110,
    'medico gastroenterologista': 2252,
    'geologo': 2141,
    'assistente de vendas': 5211,
    'escriturario de banco': 4110,
    'tecelao (tear automatico)': 7632,
    'auxiliar de enfermagem do trabalho': 3222,
    'trabalhador do acabamento de artefatos de tecidos e couros': 5162,
    'chapeador': 7233,
    'massagista': 3222,
    'montador de andaimes (edificacoes)': 7151,
    'forneiro e operador (alto-forno)': 8111,
    'motorista de onibus rodoviario': 7823,
    'gerente de marketing': 1425,
    'operador de acabamento (industria grafica)': 8213,
    'operador de urdideira': 7633,
    'esteticista': 3222,
    'cabo bombeiro militar': 5171,
    'operador de fresadora (usinagem de madeira)': 7232,
    'designer de interiores': 2624,
    'chefe de servicos bancarios': 1210,
    'cortador de pedras': 7222,
    'operador de trator florestal': 6323,
    'montador de maquinas': 7242,
    'escolhedor de papel': 7842,
    'costureiro, a maquina na confeccao em serie': 7632,
    'promotor de justica': 2622,
    'oleiro (fabricacao de telhas)': 7522,
    'quimico': 2111,
    'mecanico de manutencao de motocicletas': 9144,
    'tecnico em administracao': 4110,
    'fotografo': 2623,
    'inspetor de qualidade': 2149,
    'tecnico eletronico': 3132,
    'operador de producao (quimica, petroquimica e afins)': 8621,
    'administrador de sistemas operacionais': 2121,
    'camareiro de hotel': 5134,
    'operador de maquinas-ferramenta convencionais': 7244,
    'dirigente e administrador de organizacao da sociedade civil': 1210,
    'projetista de moveis': 2134,
    'programador de maquinas com comando numerico': 3171,
    'diretor de planejamento estrategico': 1425,
    'tecnico em turismo': 3541,
    'escriturario em estatistica': 4110,
    'operador de trator de lamina': 6323,
    'ajudante de carvoaria': 6323,
    'impressor de ofsete (plano e rotativo)': 7731,
    'bioengenheiro': 2132,
    'alfaiate': 5162,
    'condutor de veiculos de tracao animal (ruas e estradas)': 7823,
    'tecnico de laboratorio industrial': 3144,
    'lavador de roupas': 5162,
    'pesquisador em ciencias da computacao e informatica': 2122,
    'encarregado de corte na confeccao do vestuario': 5162,
    'desenhista projetista de arquitetura': 2146,
    'operador de maquina cortadora (minas e pedreiras)': 7121,
    'gerente financeiro': 1222,
    'analista de desenvolvimento de sistemas': 2124,
    'supervisor de recepcionistas': 4211,
    'chaveiro': 5199,
    'engenheiros de sistemas operacionais em computacao': 2122,
    'diretor de servicos de informatica': 1221,
    'torneiro (lavra de pedra)': 7121,
    'pesquisador de engenharia metalurgica, de minas e de materiais': 2141,
    'gerente de loja e supermercado': 1423,
    'operador de trator (minas e pedreiras)': 6323,
    'instalador-reparador de equipamentos de transmissao ': 5223,
    'tecnico mecanico (calefacao, ventilacao e refrigeracao)': 3111,
    'decorador de interiores de nivel superior': 2624,
    'maquiador': 5151,
    'produtor de especiarias': 6221,
    'encarregado de manutencao de instrumentos de medicao': 3132,
    'trabalhador da pecuaria (asininos e muares)': 6221,
    'carregador (veiculos de transportes terrestres)': 7841,
    'engenheiro civil (edificacoes)': 2142,
    'auxiliar tecnico em laboratorio de farmacia': 3222,
    'professor da area de meio ambiente': 2342,
    'acabador de calcados': 5162,
    'professor de lingua portuguesa': 2344,
    'porteiro de locais de diversao': 5171,
    'forneiro de fundicao (forno de reducao)': 7122,
    'atendente de judiciario': 4211,
    'cirurgiao dentista - odontologista legal': 2231,
    'fisico': 2111,
    'auditor-fiscal do trabalho': 2515,
    'gerente de producao e operacoes agropecuarias': 1425,
    'auxiliar de laboratorio de analises clinicas': 3241,
    'professor de nivel superior na educacao infantil': 2344,
    'despachante documentalista': 3542,
    'fisioterapeuta': 2235,
    'leiloeiro': 3541,
    'montador de equipamentos eletricos': 9511,
    'operador de central hidreletrica': 8622,
    'supervisor de tesouraria': 1210,
    'auditor-fiscal da receita federal': 2410,
    'guardador de veiculos': 5199,
    'avicultor': 6123,
    'musico regente': 2623,
    'balanceiro': 5199,
    'lavadeiro, em geral': 5199,
    'apontador de mao-de-obra': 5199,
    'oficial de justica': 3333,
    'operador de telemarketing receptivo': 4223,
    'tecnico em patologia clinica': 3241,
    'tecnico de producao em refino de petroleo': 3141,
    'mecanico montador de motores de explosao e diesel': 9113,
    'operador de caminhao (minas e pedreiras)': 7825,
    'tecnico em eletromecanica': 3132,
    'acabador de superficies de concreto': 7153,
    'comprador': 3543,
    'mecanico de refrigeracao': 9511,
    'mecanico de manutencao de compressores de ar': 9511,
    'eletricista de instalacoes (veiculos automotores )': 9121,
    'ciclista mensageiro': 5199,
    'operador de maquina de usinagem madeira, em geral': 7231,
    'tecnico em mecanica de precisao': 3141,
    'supervisor de credito e cobranca': 1222,
    'instalador de material isolante, a mao (edificacoes)': 5141,
    'eletrotenico na fabricacao de maquinas e equipamentos': 3132,
    'fiscal de patio de usina de concreto': 7121,
    'professor de educacao fisica do ensino fundamental': 2331,
    'ceramista': 7681,
    'professor de antropologia do ensino superior': 2344,
    'cozinhador de carnes': 5132,
    'desenhista tecnico naval': 2144,
    'estofador de moveis': 7632,
    'carpinteiro de obras': 7112,
    'cozinhador (conservacao de alimentos)': 5132,
    'ajustador naval (reparo e construcao)': 9113,
    'moco de conves (maritimo e fluviario)': 6410,
    'mestre de producao farmaceutica': 2232,
    'diretor de manutencao': 1425,
    'mecanico de manutencao de maquinas agricolas': 9113,
    'escrivao extra - judicial': 3331,
    'trabalhador da avicultura de corte': 6223,
    'encarregado de manutencao mecanica de sistemas operacionais': 3141,
    'locutor de radio e televisao': 2642,
    'desenhista tecnico (eletricidade e eletronica)': 2144,
    'tecnico em secretariado': 4110,
    'engenheiro quimico': 2143,
    'carvoeiro': 6323,
    'professor de alunos com deficiencia auditiva e surdos': 2331,
    'produtor de arvores frutiferas': 6221,
    'trabalhador da exploracao de madeiras tanantes': 6221,
    'masseiro (massas alimenticias)': 8484,
    'capitao da policia militar': 5162,
    'gerente de hotel': 1416,
    'operador de estacao de captacao de agua': 8621,
    'tecelao de malhas, a maquina': 7622,
    'gerente de captacao (fundos e investimentos institucionais)': 1222,
    'engenheiro eletricista': 2141,
    'classificador e empilhador de tijolos refratarios': 5141,
    'inspetor de alunos de escola privada': 2342,
    'engenheiro eletronico de manutencao': 2141,
    'praca da marinha': 6312,
    'poeta': 2623,
    'trabalhador do beneficiamento de fumo': 5143,
    'medico ortopedista e traumatologista': 2252,
    'medico psiquiatra': 2252,
    'conferente-expedidor de roupas (lavanderias)': 5199,
    'auxiliar de cartorio': 4110,
    'medico generalista': 2251,
    'marinheiro de conves (maritimo e fluviario)': 6410,
    'apresentador de eventos': 2624,
    'tecnico de comunicacao de dados': 3171,
    'mestre (industria textil e de confeccoes)': 7121,
    'operador de bate-estacas': 7153,
    'chefe de cozinha': 5131,
    'operador de forno (servicos funerarios)': 7831,
    'tecnico de alimentos': 3211,
    'reparador de aparelhos eletrodomesticos ': 9541,
    'atendente de agencia': 4211,
    'praca da aeronautica': 6312,
    'diretor de producao e operacoes da industria de transformacao': 1425,
    'operador de maquina de etiquetar': 8321,
    'diretor de operacoes de obras publica e civil': 1421,
    'analista de informacoes (pesquisador de informacoes de rede)': 2129,
    'engenheiro aeronautico': 2141,
    'operador de espelhamento': 7154,
    'prensista (operador de prensa)': 8112,
    'torneiro na usinagem convencional de madeira': 7233,
    'vigia portuario': 5174,
    'oficial da aeronautica': 6312,
    'encarregado geral de operacoes de conservacao de vias ': 5199,
    'administrador de banco de dados': 2122,
    'professor leigo no ensino fundamental': 2344,
    'ministro de culto religioso': 2631,
    'armador de estrutura de concreto armado': 7153,
    'ator': 2622,
    'gerente de cambio e comercio exterior': 1210,
    'professor de economia': 2342,
    'professor instrutor de ensino e aprendizagem em servicos': 2331,
    'operador de audio de continuidade (radio)': 3721,
    'operador de calcinacao (tratamento quimico e afins)': 7122,
    'antropologo': 2515,
    'ensaiador de danca': 2614,
    'pedreiro (chamines industriais)': 7152,
    'poceiro (edificacoes)': 7152,
    'gerente de almoxarifado': 1416,
    'coordenador pedagogico': 2344,
    'trabalhador na producao de mudas e sementes': 6221,
    'supervisor de caixas e bilheteiros (exceto caixa de banco)': 4212,
    'pescador artesanal de lagostas': 6323,
    'lubrificador industrial': 5142,
    'aplicador serigrafico em vidros': 7522,
    'adestrador de animais': 6222,
    'perito contabil': 2410,
    'gerente de projetos e servicos de manutencao': 1210,
    'relojoeiro (reparacao)': 9542,
    'afiador de ferramentas': 7223,
    'agente de patio': 5199,
    'alinhador de pneus': 9511,
    'professor de ciencias biologicas do ensino superior': 2332,
    'papiloscopista policial': 5173,
    'montador de estruturas metalicas': 7242,
    'tipografo': 2623,
    'bibliotecario': 2624,
    'bordador, a mao': 7632,
    'trabalhador na olericultura (raizes, bulbos e tuberculos)': 6221,
    'moleiro de cereais (exceto arroz)': 7621,
    'diretor administrativo': 1221,
    'trabalhador da cultura de algodao': 6223,
    'tecnico de operacoes e servicos bancarios - cambio': 4212,
    'trabalhador de extracao florestal, em geral': 6323,
    'supervisor de vendas de servicos': 1422,
    'gravador de matriz para flexografia (clicherista)': 2626,
    'professor de enfermagem do ensino superior': 2342,
    'gerente de recursos humanos': 1210,
    'desenhista tecnico (artes graficas)': 2144,
    'tecnologo em eletronica': 3132,
    'professor de tecnicas agricolas': 6120,
    'ascensorista': 5143,
    'tabeliao de notas': 5112,
    'advogado (direito civil)': 2410,
    'cirurgiao dentista - endodontista': 2232,
    'fiscal de tributos estadual': 2410,
    'auditor (contadores e afins)': 2410,
    'hidrogeologo': 2149,
    'montador de bicicletas': 9143,
    'coronel bombeiro militar': 5162,
    'analista de seguros (tecnico)': 3542,
    'operador eletromecanico': 3143,
    'operador de maquina de fabricar papel e papelao': 8331,
    'criador em pecuaria polivalente': 6223,
    'cirurgiao dentista de saude coletiva': 2232,
    'instalador de isolantes termicos (refrigeracao e climatizacao)': 9541,
    'operador de atendimento aeroviario': 7825,
    'produtor de arroz': 6223,
    'eletricista de manutencao de linhas eletricas': 9511,
    'retalhador de carne': 8484,
    'instalador-reparador de redes e cabos telefonicos': 5231,
    'monitor de teleatendimento': 4223,
    'salva-vidas': 5199,
    'tecnico quimico': 3241,
    'controlador de entrada e saida': 5199,
    'operador de acabamento de pecas fundidas': 7244,
    'juiz de direito': 2410,
    'operador de guindaste movel': 7822,
    'bombeiro de aerodromo': 5172,
    'medico angiologista': 2252,
    'analista de credito (instituicoes financeiras)': 2522,
    'tecnico mecanico (motores)': 9144,
    'auxiliar de biblioteca': 2614,
    'ajustador mecanico (usinagem em bancada)': 9113,
    'apresentador de programas de radio': 2613,
    'comissario de voo': 5111,
    'operador de projetor cinematografico': 3421,
    'medico sanitarista': 2252,
    'diretor de producao e operacoes em empresa agropecuaria': 1012,
    'engenheiro agricola': 2141,
    'montador de equipamento de levantamento': 9141,
    'desenhista tecnico mecanico': 3182,
    'narrador em programas de radio e televisao': 2613,
    'oficial da marinha': 6112,
    'mecanico de voo': 9144,
    'guia de turismo': 3541,
    'relacoes publicas': 2521,
    'operador de britadeira (tratamentos quimicos e afins)': 7121,
    'operador de pa carregadeira': 7151,
    'gerente de bar': 1415,
    'tecnico de obras civis': 3143,
    'primeiro tenente de policia militar': 5172,
    'matematico': 2121,
    'medico legista': 2252,
    'operador de equipamento para resfriamento': 9512,
    'carpinteiro naval (construcao de pequenas embarcacoes)': 7542,
    'afinador de instrumentos musicais': 2626,
    'mestre serralheiro': 7242,
    'supervisor de telemarketing e atendimento': 1421,
    'major da policia militar': 5172,
    'engenheiro de manutencao de telecomunicacoes': 2145,
    'tecnico em estruturas metalicas': 3132,
    'plataformista (petroleo)': 8622,
    'compositor': 2631,
    'gerente de producao e operacoes da construcao civil': 1210,
    'tecnico mecanico na fabricacao de ferramentas': 3111,
    'classificador de madeira': 7824,
    'diretor de servicos de saude': 1421,
    'instalador eletricista (tracao de veiculos)': 9511,
    'domador de animais (circense)': 6324,
    'supervisor da industria de minerais nao metalicos ': 1422,
    'economista financeiro': 2516,
    'auxiliar de maquinista de trem': 6114,
    'pintor de estruturas metalicas': 7152,
    'gerente de produtos bancarios': 1210,
    'diretor de producao e operacoes de turismo': 1210,
    'operador de mensagens de telecomunicacoes (correios)': 3171,
    'cronoanalista': 2149,
    'operador de torno automatico (usinagem de madeira)': 7223,
    'supervisor de manutencao (eletromecanica)': 3132,
    'gerente de producao de tecnologia da informacao': 1221,
    'diretor de cinema': 2624,
    'pedreiro (mineracao)': 7102,
    'operador de camera de televisao': 3721,
    'tecnico de desporto individual e coletivo (exceto futebol)': 2242,
    'montador de maquinas, motores e acessorios': 9101,
    'trabalhador na operacao de sistema de irrigacao localizada ': 6221,
    'recepcionista de banco': 4211,
    'gerente de compras': 1423,
    'operador de betoneira': 7152,
    'governanta de hotelaria': 5133,
    'sinaleiro (ponte-rolante)': 7155,
    'tecnico de operacoes e servicos bancarios - credito rural': 4212,
    'cenotecnico (cinema, video, televisao, teatro e espetaculos)': 3742,
    'escrevente': 4122,
    'trabalhador da caprinocultura': 6221,
    'medico ginecologista e obstetra': 2252,
    'professor de estatistica (no ensino superior)': 2343,
    'operador de alambique de funcionamento continuo ': 8621,
    'amostrador de minerios': 7121,
    'guincheiro (construcao civil)': 7151,
    'mecanico de manutencao de veiculos ferroviarios': 9144,
    'estatistico': 2123,
    'operador de veiculos subaquaticos controlados remotamente': 3413,
    'medico pediatra': 2252,
    'montador de equipamentos eletricos ': 9511,
    'carpinteiro de formas para concreto': 7152,
    'mecanico de manutencao de maquinas-ferramentas': 9144,
    'professor de artes no ensino medio': 2332,
    'criador de bovinos (leite)': 6223,
    'cortador de laminados de madeira': 7631,
    'operador de instalacao de ar-condicionado': 9532,
    'medico dermatologista': 2252,
    'juiz do trabalho': 2410,
    'pesquisador em medicina veterinaria': 2115,
    'professor de musica no ensino superior': 2332,
    'engenheiro de minas (lavra a ceu aberto)': 2142,
    'inspetor de soldagem': 2146,
    'operador de desempenadeira na usinagem convencional': 7223,
    'professor de nivel medio no ensino profissionalizante': 2332,
    'avaliador de bens moveis': 3543,
    'condutor de processos robotizados de pintura': 5141,
    'piloto de aeronaves': 6123,
    'arquivista': 2612,
    'atuario': 2123,
    'criador de suinos': 6223,
    'motorista operacional de guincho': 7822,
    'artista de circo (outros)': 2626,
    'tecnico de estradas': 3143,
    'chefe de servico de transporte rodoviario ': 1342,
    'artifice do couro': 7233,
    'produtor de radio': 2614,
    'agente de estacao (ferrovia e metro)': 5172,
    'tecelao (tear mecanico liso)': 7612,
    'cumim': 5134,
    'modelo artistico': 5161,
    'analista de recursos humanos': 2522,
    'instalador de tubulacoes': 7151,
    'operador de impermeabilizador de tecidos': 8411,
    'modelador de metais (fundicao)': 7244,
    'piloto comercial (exceto linhas aereas)': 6123,
    'preparador de calcados': 7231,
    'caldeireiro (chapas de cobre)': 7241,
    'tecnico em manutencao de instrumentos de medicao e precisao': 3142,
    'colorista de papel': 7515,
    'modelo publicitario': 5161,
    'produtor de cereais de inverno': 6222,
    'supervisor de reparos linhas ferreas': 9131,
    'tecnico de manutencao eletrica de maquina': 3132,
    'filosofo': 2313,
    'pesquisador em ciencias sociais e humanas': 2013,
    'medico alergista e imunologista': 2252,
    'corretor de valores, ativos financeiros e derivativos': 3541,
    'produtor de algodao': 6121,
    'tecnico de manutencao de sistemas e instrumentos': 3132,
    'cartazeiro': 2621,
    'tecnico em operacao de equipamentos para televisao ': 3741,
    'tecnico de controle de meio ambiente': 3548,
    'instrumentador cirurgico': 3222,
    'operador de camaras frias': 7842,
    'operador de maquina extrusora de varetas e tubos de vidro': 8623,
    'conservador de via permanente (trilhos)': 7831,
    'tecnico de utilidade (producao e distribuicao de vapor)': 3132,
    'supervisor de transportes': 1422,
    'curtidor (couros e peles)': 7521,
    'forjador': 7241,
    'operador de cabine de laminacao (fio-maquina)': 8623,
    'tecnico em higiene dental': 3221,
    'mecanico de manutencao de redutores': 9144,
    'tecnico em madeira': 3143,
    'avaliador fisico': 2145,
    'montador de equipamentos eletricos (centrais eletricas)': 9511,
    'desenhista tecnico (construcao civil)': 3182,
    'vidraceiro (vitrais)': 7243,
    'cenografo carnavalesco e festas populares': 2621,
    'operador de transmissor de radio': 3171,
    'medico de saude da familia': 2252,
    'operador de retificadora com comando numerico': 7122,
    'encarregado de manutencao eletrica de veiculos': 9131,
    'arquiteto urbanista': 2145,
    'foguista (locomotivas a vapor)': 9131,
    'pesquisador de engenharia civil': 2145,
    'medico em medicina intensiva': 2252,
    'operador de linha de montagem (aparelhos eletricos)': 7641,
    'cartonageiro, a maquina': 8332,
    'trabalhador no cultivo de arvores frutiferas': 6222,
    'editor de tv e video': 2613,
    'medico em radiologia e diagnostico por imagem': 2252,
    'aplicador de asfalto impermeabilizante (coberturas)': 7151,
    'neuropsicologo': 2516,
    'assistente comercial de seguros': 3543,
    'mestre de ferramentaria': 7241,
    'compensador de banco': 4141,
    'diretor financeiro': 1342,
    'supervisor de coletadores de apostas e de jogos': 1422,
    'passador de roupas, a mao': 5134,
    'apicultor': 6223,
    'enologo': 6121,
    'agente de protecao de aeroporto': 5172,
    'professor de lingua portuguesa do ensino fundamental': 2332,
    'lixador de couros e peles': 7631,
    'supervisor de montagem e instalacao eletroeletronica': 3132,
    'professor de literatura portuguesa': 2332,
    'montador de instrumentos de precisao': 9521,
    'montador de veiculos (linha de montagem)': 9113,
    'operador de instalacao de refrigeracao': 9541,
    'trabalhador da pecuaria (equinos)': 6223,
    'sacristao': 2632,
    'auxiliar de estatistica': 4152,
    'mecanico de manutencao de maquinas operatrizes': 9144,
    'pirotecnico': 7611,
    'maquetista na marcenaria': 7222,
    'professor pratico no ensino profissionalizante': 2332,
    'controlador de servicos de maquinas e veiculos': 5211,
    'operador de moenda na fabricacao de acucar': 8621,
    'cozinheiro industrial': 5132,
    'professor de contabilidade': 2341,
    'operador de plaina desengrossadeira': 7122,
    'medico oftalmologista': 2252,
    'operador de instalacao de extracao de gases': 8622,
    'preparador de esmaltes (ceramica)': 8331,
    'estenotipista': 2511,
    'supervisor de exploracao pecuaria': 6221,
    'operador de draga': 8111,
    'secador de madeira': 7711,
    'professor de nutricao': 2252,
    'montador de equipamentos eletronicos (estacao de radio, tv)': 9511,
    'dirigente de partido politico': 1311,
    'procurador da republica': 2410,
    'preparador de maquinas-ferramenta': 7242,
    'diretor de operacoes de correios': 1311,
    'dirigentes de entidades de trabalhadores': 1311,
    'tecnico em agrimensura': 3143,
    'dirigente e administrador de organizacao religiosa': 1311,
    'programador de internet': 2133,
    'decorador de ceramica': 2621,
    'operador de sistemas de prova (analogico e digital)': 3171,
    'eletromecanico de manutencao de elevadores': 3134,
    'professor de filosofia do ensino superior': 2313,
    'operador de negocios': 3541,
    'trefilador de borracha': 7623,
    'classificador de fibras texteis': 7641,
    'tecnico textil': 3143,
    'sondador (pocos de petroleo e gas)': 7101,
    'supervisor de lavanderia': 5162,
    'professor de linguas estrangeiras modernas': 2332,
    'maitre': 5134,
    'medico citopatologista': 2252,
    'operador de maquina de secar celulose': 8631,
    'engenheiro de controle e automacao': 2143,
    'operador de maquina perfuradora (minas e pedreiras)': 7122,
    'datilografo': 4110,
    'operador de equipamentos de preparacao de areia': 7122,
    'operador de computador (inclusive microcomputador)': 3171,
    'laboratorista fotografico': 3241,
    'tecelao de tapetes, a maquina': 7612,
    'consultor contabil (tecnico)': 2522,
    'engenheiro de seguranca do trabalho': 2141,
    'pesquisador de clinica medica': 2015,
    'operador de cristalizacao na refinacao de acucar': 8621,
    'pesquisador em ciencias agronomicas': 2015,
    'professor de filosofia no ensino medio': 2332,
    'ancora de radio e televisao': 2613,
    'auxiliar de banco de sangue': 3222,
    'arquivista pesquisador (jornalismo)': 2623,
    'tecnico em programacao visual': 3742,
    'mestre (construcao naval)': 7102,
    'tecnico em petroquimica': 3143,
    'gelador industrial': 8412,
    'teologo': 2513,
    'dancarino tradicional': 5198,
    'tecelao (tear jacquard)': 7612,
    'medico cirurgiao plastico': 2252,
    'diretor de producao e operacoes em empresa florestal': 1321,
    'supervisor de producao da industria alimenticia': 1422,
    'destrocador de pedra': 7122,
    'analista de pesquisa de mercado': 2533,
    'desenhista detalhista': 3182,
    'engenheiro de producao': 2143,
    'supervisor de manutencao eletromecanica': 3132,
    'arquiteto de patrimonio': 2145,
    'professor de historia do ensino fundamental': 2332,
    'pratico de portos da marinha mercante': 6341,
    'limpador de fachadas': 5141,
    'professor de astronomia (ensino superior)': 2311,
    'profissional do sexo': 5198,
    'operador de rebobinadeira na fabricacao de papel e papelao': 7642,
    'operador de processo de moagem': 8111,
    'medico do trabalho': 2252,
    'trabalhador da cultura de erva-mate': 6222,
    'taifeiro': 5134,
    'cortador de calcados, a maquina (exceto solas e palmilhas)': 7243,
    'trabalhador da exploracao de acai': 6222,
    'engenheiro projetista de telecomunicacoes': 2143,
    'tecnico em optica': 3132,
    'medico cirurgiao pediatrico': 2252,
    'diretor de instituicao educacional da area privada': 1342,
    'operador de maquinas do acabamento de couros e peles': 7631,
    'telhador (telhas de argila e materias similares)': 7151,
    'operador de inspecao de qualidade': 8413,
    'arbitro desportivo': 3751,
    'acrobata': 5198,
    'operador de jato abrasivo': 7642,
    'cenografo de eventos': 2621,
    'instrutor de voo': 3313,
    'tecnico em piscicultura': 3143,
    'controlador de trafego aereo': 3324,
    'acabador de embalagens (flexiveis e cartotecnicas)': 8213,
    'operador de abertura (fiacao)': 8212,
    'tecnico de enfermagem do trabalho': 3222,
    'embalsamador': 3221,
    'mestre de siderurgia': 7102,
    'tecnico de planejamento de producao': 3143,
    'enfermeiro veterinario': 2234,
    'montador de artefatos de couro (exceto roupas e calcados)': 7243,
    'tecelao (tear mecanico, exceto jacquard)': 7612,
    'astrologo': 2513,
    'analista de exportacao e importacao': 2533,
    'preparador de tintas': 7631,
    'engenheiro agrimensor': 2145,
    'gerente de pesquisa e desenvolvimento (p&d)': 1210,
    'capitao bombeiro militar': 7161,
    'costurador de calcados, a maquina (exceto solas e palmilhas)': 7243,
    'instalador de cortinas e persianas, portas sanfonadas e boxe': 5142,
    'presidiario': 5198,
    'editor de midia eletronica': 2623,
    'tecnico em sonorizacao': 3741,
    'professor de matematica no ensino medio': 2332,
    'operador de forno (fabricacao de paes, biscoitos e similares)': 8481,
    'trabalhador polivalente do curtimento de couros e peles': 7631,
    'economista do setor publico': 2512,
    'confeccionador de artefatos de couro (exceto sapatos)': 7243,
    'moleiro de minerios': 7102,
    'operador de maquina de fabricacao de cosmeticos': 8621,
    'oficial do registro civil de pessoas naturais': 3211,
    'trabalhador no cultivo de plantas ornamentais': 6222,
    'finalizador de video': 2613,
    'camareira de teatro': 5143,
    'trabalhador na fabricacao de produtos abrasivos': 8411,
    'instalador de sistemas eletroeletronicos de seguranca': 5173,
    'forneiro (materiais de construcao)': 8622,
    'entrevistador de pesquisas de mercado': 3543,
    'cozinheiro de embarcacoes': 5132,
    'mestre (industria de madeira e mobiliario)': 7102,
    'operador de extrusora (quimica, petroquimica e afins)': 8623,
    'modelista de calcados': 7242,
    'viveirista florestal': 6222,
    'operador de motoniveladora (extracao de minerais solidos)': 7112,
    'dietista': 2232,
    'escritor de ficcao': 2623,
    'operador de maquina de cilindrar chapas': 7122
}

In [ ]:
df["OCUP"].isnull().sum()

52217

In [ ]:
# Substituição das linhas nulas por -1
df['OCUP'] = df['OCUP'].fillna("desconhecido")

In [ ]:
df["OCUP"] = df.apply(lambda row: row['OCUP'].lower() if isinstance(row['OCUP'], str) else row['OCUP'], axis=1)

In [ ]:
 df['OCUP'] = df['OCUP'].map(ocup_map)
 df['OCUP'] = df['OCUP'].fillna(-1)

In [ ]:
df.head()

,estado,DTOBITO,DTNASC,SEXO,RACACOR,ESTCIV,ESC,OCUP,CAUSABAS
0,AC,2010-01-31,1999-11-09,Masculino,Parda,1.0,2.000000,-1.0,X780
1,AC,2010-02-07,1985-09-23,Masculino,Parda,1.0,2.000000,-1.0,X780
2,AC,2010-02-19,1995-08-06,Masculino,Parda,1.0,2.000000,-1.0,X780
3,AC,2010-04-20,1983-12-25,Feminino,Parda,2.0,2.000000,-1.0,X780
4,AC,2010-07-24,1982-05-13,Masculino,Parda,2.0,3.229998,-1.0,X720


### Raça/Cor

* No processo de EDA da base, identificou-se que essa feature é representada pela seguinte legenda:
  * 1: Branca
  * 2: Preta
  * 3: Amarela
  * 4: Parda
  * 5: Indígena
  * 9: Ignorado

* Para essa feature, adotou-se a estratégia de considerar todos os casos nulos como ignorados (valor igual a 9).

In [ ]:
# Distribuição dos valores nulos e não-nulos na coluna
df['RACACOR'].isnull().sum(), df['RACACOR'].value_counts()

(3171,
 RACACOR
 Branca      56075
 Parda       45695
 Preta        5929
 Indígena     1187
 Amarela       434
 Name: count, dtype: int64)

In [ ]:
 df['RACACOR'] = df['RACACOR'].map({'Branca': 1, 'Preta': 2, 'Amarela': 3, 'Parda': 4, 'Indígena': 5})

In [ ]:
# Substituição das linhas nulas por 9 (casos ignorados)
df['RACACOR'] = df['RACACOR'].fillna(9)

In [ ]:
# Distribuição dos valores nulos e não-nulos na coluna
df['RACACOR'].isnull().sum(), df['RACACOR'].value_counts()

(0,
 RACACOR
 1.0    56075
 4.0    45695
 2.0     5929
 9.0     3171
 5.0     1187
 3.0      434
 Name: count, dtype: int64)

### Naturalidade

* No processo de EDA da base, identificou-se que essa feature representa a naturalidade do falecido, conforme a tabela de países. Se for brasileiro, porém, o primeiro dígito contém 8 e os demais o código da UF de naturalidade.

* Dito isso, a estratégia adotada foi a de substituir os valores nulos por um valor que já não estivesse presente na coluna e que pudesse caracterizar uma naturalidade desconhecida, -1 foi o valor escolhido.

In [ ]:
df['estado'].isnull().sum(), df['estado'].value_counts()

(0,
 estado
 SP    21970
 MG    13541
 RS    11860
 PR     7252
 SC     6349
 CE     5805
 RJ     5373
 BA     5142
 GO     4453
 PE     3583
 PI     2710
 MA     2676
 PA     2640
 MS     2290
 AM     2145
 PB     2009
 ES     1899
 MT     1835
 RN     1733
 DF     1490
 SE     1213
 AL     1159
 TO     1030
 RO      995
 AC      508
 AP      429
 RR      402
 Name: count, dtype: int64)

In [ ]:
 estados_map = {'AC': 12, 'AL': 27, 'AP': 16, 'AM': 13, 'BA': 29, 'CE': 23, 'DF': 53, 'ES': 32, 'GO': 52, 'MA': 21, 'MT': 51,
       'MS': 50, 'MG': 31, 'PA': 15, 'PB': 25, 'PR': 41, 'PE': 26, 'PI': 22, 'RJ': 33, 'RN': 24, 'RS': 43, 'RO': 11,
       'RR': 14, 'SC': 42, 'SP': 35, 'SE': 28, 'TO': 17}

 df['estado'] = df['estado'].map(estados_map)

In [ ]:
df = df.rename(columns={'estado': 'NATURAL'})

### Idade

* No processo de EDA da base, identificou-se que essa feature representa a idade, composta de dois subcampos. O primeiro, de 1 dígito, indica a unidade da idade. O segundo, de dois dígitos, indica a quantidade de unidades conforme a legenda abaixo:
  * Primeiro dígito igual a 0: Idade ignorada
  * Primeiro dígito igual a 1: Unidade em horas, o segundo subcampo varia de 01 a 23
  * Primeiro dígito igual a 2: Unidade em dias, o segundo subcampo varia de 01 a 29
  * Primeiro dígito igual a 3: Unidade em meses, o segundo subcampo varia de 01 a 11
  * Primeiro dígito igual a 4: Unidade em anos, o segundo subcampo varia de 00 a 99
  * Primeiro dígito igual a 5: Unidade em anos (mais de 100 anos), o segundo subcampo varia de 0 a 99
* Exemplo:
  * 410: 10 anos
  * 457: 57 anos
  * 505: 105 anos

* Para essa feature, primeiramente se descartou idades que não estão em anos, pois não fazem sentido para o objetivo do projeto. Logo após substituiu-se os valores nulos por 0 (ignorados), por fim trocou-se a notação presente pela padrão de idade em anos e depois substituiu-se os valores ignorados pela média da coluna por meio do uso de um imputer.

In [ ]:
# Conversão das colunas para objetos de data
df['DTOBITO'] = pd.to_datetime(df['DTOBITO'], format='%Y-%m-%d', errors='coerce')
df['DTNASC'] = pd.to_datetime(df['DTNASC'], format='%Y-%m-%d', errors='coerce')

In [ ]:
# Calculo da diferença da data de óbito e da data de nascimento para encontrar a idade
delta_date = (df['DTOBITO'] - df['DTNASC'])
df['IDADE'] = [date.days / 365.25 for date in delta_date]

In [ ]:
df = df.drop(columns=['DTOBITO', 'DTNASC'])

In [ ]:
df.head(3)

,NATURALIDADE,SEXO,RACACOR,ESTCIV,ESC,OCUP,CAUSABAS,IDADE
0,12,Masculino,4.0,1.0,2.0,-1.0,X780,10.228611
1,12,Masculino,4.0,1.0,2.0,-1.0,X780,24.375086
2,12,Masculino,4.0,1.0,2.0,-1.0,X780,14.540726


In [ ]:
# Distribuição dos valores nulos e não-nulos na coluna
df['IDADE'].isnull().sum(), df['IDADE'].value_counts()

(11742,
 IDADE
 20.199863    17
 41.336071    16
 37.262149    16
 19.154004    16
 30.959617    16
              ..
 59.980835     1
 82.392882     1
 83.381246     1
 78.694045     1
 72.640657     1
 Name: count, Length: 25145, dtype: int64)

In [ ]:
# Troca os valores nulos por 0 (ignorados) e converte para int
df['IDADE'] = df['IDADE'].fillna(0.0)

In [ ]:
# Instanciação da classe para substituir os valores nulos
imputer = SimpleImputer(missing_values=0, strategy='mean')

In [ ]:
# Usa o imputer para substituir os valores nulos pela média
imputer = imputer.fit(df[['IDADE']])
df['IDADE'] = imputer.transform(df[['IDADE']])

In [ ]:
# Distribuição dos valores nulos e não-nulos na coluna
df['IDADE'].isnull().sum(), df['IDADE'].value_counts()

(0,
 IDADE
 41.677684    11742
 20.199863       17
 41.336071       16
 19.154004       16
 30.959617       16
              ...  
 59.980835        1
 82.392882        1
 83.381246        1
 78.694045        1
 72.640657        1
 Name: count, Length: 25146, dtype: int64)

### Sexo

* No processo de EDA da base, identificou-se que essa feature é representada pela seguinte legenda:
  * 0: Ignorado
  * 1: Masculino
  * 2: Feminino

* Essa feature não apresentava valores nulos, então somente foi realizada a remoção de outliers.

In [ ]:
df['SEXO'].value_counts(), df['SEXO'].isnull().sum()

(SEXO
 Masculino    88394
 Feminino     24075
 Name: count, dtype: int64,
 22)

In [ ]:
 df['SEXO'] = df['SEXO'].map({'Masculino': 1, 'Feminino': 2})

In [ ]:
df['SEXO'].value_counts(), df['SEXO'].isnull().sum()

(SEXO
 1.0    88394
 2.0    24075
 Name: count, dtype: int64,
 22)

### Causa Base

In [ ]:
df["CAUSABAS"] = 1

In [ ]:
df = df.rename(columns={"CAUSABAS": "SUICIDIO"})

In [ ]:
df

,NATURALIDADE,SEXO,RACACOR,ESTCIV,ESC,OCUP,SUICIDIO,IDADE
0,12,1.0,4.0,1.0,2.000000,-1.0,1,10.228611
1,12,1.0,4.0,1.0,2.000000,-1.0,1,24.375086
2,12,1.0,4.0,1.0,2.000000,-1.0,1,14.540726
3,12,2.0,4.0,2.0,2.000000,-1.0,1,26.318960
4,12,1.0,4.0,2.0,3.229998,-1.0,1,28.197125
...,...,...,...,...,...,...,...,...
112486,17,2.0,4.0,1.0,3.000000,5134.0,1,28.095825
112487,17,2.0,4.0,1.0,3.000000,-1.0,1,52.605065
112488,17,1.0,1.0,2.0,4.000000,5134.0,1,42.047912
112489,17,1.0,4.0,2.0,3.229998,-1.0,1,41.420945


## Tratamento de features categóricas

### One Hot Enconding

* Da forma que a feature SEXO está, o modelo pode acabar por inferir alguma relação ordinal entre os valores 'masculino' (1) e 'feminino' (2), o que não existe na realidade. Diferentemente de uma feature como escolaridade, por exemplo, em que quanto maior o valor numérico maior a escolaridade daquele indivíduo.
* Para contornar isso, foi utilizada uma estratégia de one hot enconding, na qual essa coluna foi subdividida em duas, cada uma representando um dos estados possíveis (masculino ou feminino) podendo assumir os valores "True" ou "False".

In [ ]:
# Os valores ignorados (iguais a 0) são muito poucos
df['SEXO'].value_counts()

SEXO
1.0    88394
2.0    24075
Name: count, dtype: int64

In [ ]:
# Criação e concatenação das novas colunas, assim como remoção da coluna antiga para a feature SEXO
one_hot_sexo = pd.get_dummies(df['SEXO'], prefix='SEXO')
df = pd.concat([df, one_hot_sexo], axis=1)
df = df.rename(columns={'SEXO_1.0' : 'MASCULINO', 'SEXO_2.0' : 'FEMININO'}).drop(columns=['SEXO'])

In [ ]:
df.dropna(inplace=True)

In [ ]:
# Como o dataframe ficou
df

,NATURALIDADE,RACACOR,ESTCIV,ESC,OCUP,SUICIDIO,IDADE,MASCULINO,FEMININO
0,12,4.0,1.0,2.000000,-1.0,1,10.228611,True,False
1,12,4.0,1.0,2.000000,-1.0,1,24.375086,True,False
2,12,4.0,1.0,2.000000,-1.0,1,14.540726,True,False
3,12,4.0,2.0,2.000000,-1.0,1,26.318960,False,True
4,12,4.0,2.0,3.229998,-1.0,1,28.197125,True,False
...,...,...,...,...,...,...,...,...,...
112486,17,4.0,1.0,3.000000,5134.0,1,28.095825,False,True
112487,17,4.0,1.0,3.000000,-1.0,1,52.605065,False,True
112488,17,1.0,2.0,4.000000,5134.0,1,42.047912,True,False
112489,17,4.0,2.0,3.229998,-1.0,1,41.420945,True,False


## Concatenação dos dataframes

In [ ]:
old_df = pd.read_parquet("../data/processed/sim_2006_2017_pre_processed.parquet")
old_df

,ESC,ESTCIV,IDADE,NATURAL,OCUP,RACACOR,MASCULINO,FEMININO,SUICIDIO
0,-1.732328,0.809986,-0.187545,-1,0.844239,-0.484485,True,False,0
1,-0.226709,-0.329698,0.753377,-1,-1.066134,3.994021,True,False,0
2,-0.226709,0.240144,1.380658,-1,-0.337181,-0.484485,False,True,0
3,-0.226709,-0.899540,1.171564,-2,-1.066134,-0.484485,True,False,0
4,-1.732328,0.240144,0.701103,-1,-1.587212,-0.484485,False,True,0
...,...,...,...,...,...,...,...,...,...
3045770,-0.979518,-0.899540,-1.337560,35,0.844239,3.994021,False,True,0
3045771,-0.000199,3.659197,-0.082998,35,-1.587212,-0.484485,True,False,0
3045772,-0.226709,-0.899540,-0.605732,29,0.844239,1.194955,True,False,0
3045773,-0.226709,0.240144,1.067017,35,0.844239,-0.484485,True,False,0


In [ ]:
suicidio_0 = old_df[old_df['SUICIDIO'] == 0]
suicidio_1 = old_df[old_df['SUICIDIO'] == 1]

suicidio_0 = suicidio_0.sample(n=len(suicidio_1), replace=False)

old_df = pd.concat((suicidio_0, suicidio_1), ignore_index=True).reset_index(drop=True)
old_df

,ESC,ESTCIV,IDADE,NATURAL,OCUP,RACACOR,MASCULINO,FEMININO,SUICIDIO
0,-0.979518,0.240144,0.544283,35,0.844237,-0.484485,False,True,0
1,-0.000199,-0.329698,-0.867099,-1,-1.587212,0.075329,True,False,0
2,-1.732328,-0.329698,-0.605732,35,0.844237,-0.484485,False,True,0
3,-0.226709,-0.329698,0.544283,-1,-1.243376,-0.484485,True,False,0
4,-0.226709,0.240144,0.596557,35,0.844237,-0.484485,False,True,0
...,...,...,...,...,...,...,...,...,...
47159,-0.226709,-0.899540,-2.435302,35,-0.338883,1.194955,False,True,1
47160,2.031719,-0.899540,-1.912568,35,-0.822745,1.194955,True,False,1
47161,-0.226709,-0.329698,0.962471,35,0.844239,1.194955,True,False,1
47162,-1.732328,-0.899540,-0.762552,35,-1.587212,-0.484485,True,False,1


In [ ]:
balanced_df = pd.concat((old_df, df),  ignore_index=True).reset_index(drop=True)
balanced_df

,ESC,ESTCIV,IDADE,NATURAL,OCUP,RACACOR,MASCULINO,FEMININO,SUICIDIO
0,-0.979518,0.240144,0.544283,35,0.844237,-0.484485,False,True,0
1,-0.000199,-0.329698,-0.867099,-1,-1.587212,0.075329,True,False,0
2,-1.732328,-0.329698,-0.605732,35,0.844237,-0.484485,False,True,0
3,-0.226709,-0.329698,0.544283,-1,-1.243376,-0.484485,True,False,0
4,-0.226709,0.240144,0.596557,35,0.844237,-0.484485,False,True,0
...,...,...,...,...,...,...,...,...,...
159650,3.000000,1.000000,28.095825,17,5134.000000,4.000000,False,True,1
159651,3.000000,1.000000,52.605065,17,-1.000000,4.000000,False,True,1
159652,4.000000,2.000000,42.047912,17,5134.000000,1.000000,True,False,1
159653,3.229998,2.000000,41.420945,17,-1.000000,4.000000,True,False,1


## Normalização

* As diferentes features numéricas da base apresentam escalas diferentes. Por exemplo, para a coluna ESC o valor 5 é o maior alto existente, enquanto para IDADE esse valor é 129. Para resolver esse problema, é importante fazer uma normalização dos dados.

In [ ]:
std = StandardScaler()
columns = ['ESC', 'ESTCIV', 'IDADE', 'OCUP', 'RACACOR']
balanced_df[columns] = std.fit_transform(balanced_df[columns])

In [ ]:
balanced_df = balanced_df[['ESC', 'ESTCIV', 'IDADE', 'NATURAL', 'OCUP', 'RACACOR', 'MASCULINO', 'FEMININO', 'SUICIDIO']]
balanced_df

,ESC,ESTCIV,IDADE,NATURAL,OCUP,RACACOR,MASCULINO,FEMININO,SUICIDIO
0,-1.948255,-0.594321,-1.145205,35,-0.478010,-1.148027,False,True,0
1,-1.371930,-0.818146,-1.201671,-1,-0.479023,-0.866007,True,False,0
2,-2.391280,-0.818146,-1.191214,35,-0.478010,-1.148027,False,True,0
3,-1.505230,-0.818146,-1.145205,-1,-0.478879,-1.148027,True,False,0
4,-1.505230,-0.594321,-1.143114,35,-0.478010,-1.148027,False,True,0
...,...,...,...,...,...,...,...,...,...
159650,0.393675,-0.295861,-0.042932,17,1.659410,1.111148,False,True,1
159651,0.393675,-0.295861,0.937625,17,-0.478778,1.111148,False,True,1
159652,0.982170,0.096923,0.515258,17,1.659410,-0.400180,True,False,1
159653,0.529027,0.096923,0.490174,17,-0.478778,1.111148,True,False,1


## Salvamento dos dados pré-processados

In [543]:
balanced_df.to_parquet("../data/processed/sim_2006_2017_balanced.parquet")